In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/macropy/core/hquotes.py
QUANTAXIS>> Finding macros in 'macropy.core.hquotes'
QUANTAXIS>> Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/macropy/core/quotes.py
QUANTAXIS>> Finding macros in 'macropy.core.quotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/macropy/core/failure.py
QUANTAXIS>> Finding macros in 'macropy.core.failure'
QUANTAXIS>> Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/cffi/api.py
QUANTAXIS>> Fi

In [2]:
candles = qa.QA_fetch_stock_day_adv('000001', start='2017-01-01', end='2018-05-28').to_qfq()

In [4]:
candles.data.tail(5)

,,code,open,high,low,close,volume,amount,date,high_limit,low_limit,next_day_high_limit,next_day_low_limit,preclose,adj
date,code,,,,,,,,,,,,,,
2018-05-18,000001,000001,10.81,10.97,10.76,10.96,578384.0,6.282502e+08,2018-05-18,11.90,11.90,12.06,9.86,10.82,1.0
2018-05-21,000001,000001,11.07,11.11,10.93,10.95,763533.0,8.387506e+08,2018-05-21,12.06,12.06,12.05,9.86,10.96,1.0
2018-05-22,000001,000001,10.95,10.98,10.79,10.86,589871.0,6.398152e+08,2018-05-22,12.05,12.05,11.95,9.77,10.95,1.0
2018-05-23,000001,000001,10.82,10.82,10.62,10.65,991016.0,1.058588e+09,2018-05-23,11.95,11.95,11.72,9.59,10.86,1.0
2018-05-24,000001,000001,10.66,10.68,10.58,10.61,688152.0,7.306788e+08,2018-05-24,11.72,11.72,11.67,9.55,10.65,1.0


In [6]:
import matplotlib.pyplot as plt
import matplotlib.finance as mpf

ModuleNotFoundError: No module named 'matplotlib.finance'

In [7]:
from pyecharts import Grid, Bar, Line, Kline, Overlap

In [10]:
ochl = candles.data[['open', 'close', 'high', 'low']]
ochl_tolist = [ochl.ix[i].tolist() for i in range(len(ochl))]

kline = Kline()
kline.add("日K", ochl.reset_index()['date'], ochl_tolist, is_datazoom_show=True)

overlap = Overlap()
overlap.add(kline)
# overlap.add(line)
overlap



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [12]:
ochl_tolist[:3]

[[8.978789422143866, 9.028069276271987, 9.047781217923236, 8.959077480492617],
 [9.018213305446363, 9.028069276271987, 9.047781217923236, 9.008357334620738],
 [9.037925247097611, 9.037925247097611, 9.047781217923236, 9.018213305446363]]

In [16]:
raw = ochl_tolist
raw_date = ochl.reset_index()['date'].tolist()
mrg = []
mrg_date = []
mrg_cnt = []
raw_idx = 0
mrg_idx = 0
while True:
    if raw_idx >= len(raw):
        break
    if raw_idx == 0:
        mrg.append(raw[raw_idx])
        mrg_date.append(raw_date[raw_idx])
        mrg_cnt.append(1)
        raw_idx += 1
        continue
    # T+1 cover T  or T cover T+1
    if (raw[raw_idx][3] <= mrg[mrg_idx][3] and raw[raw_idx][2] >= mrg[mrg_idx][2]) or (raw[raw_idx][3] >= mrg[mrg_idx][3] and raw[raw_idx][2] <= mrg[mrg_idx][2]):
        if mrg_idx == 0 or mrg[mrg_idx-1][2] < mrg[mrg_idx][2]:
            mrg[mrg_idx][0] = max(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = max(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = max(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = max(mrg[mrg_idx][3], raw[raw_idx][3])
        else:
            mrg[mrg_idx][0] = min(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = min(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = min(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = min(mrg[mrg_idx][3], raw[raw_idx][3])
        mrg_date[mrg_idx] = raw_date[raw_idx]
        mrg_cnt[mrg_idx] += 1
    else:
        mrg.append(raw[raw_idx])
        mrg_date.append(raw_date[raw_idx])
        mrg_cnt.append(1)
        mrg_idx += 1
    raw_idx += 1

    